Imports

In [6]:
# implementation according to https://link.springer.com/content/pdf/10.1007/s40092-017-0204-z.pdf
import copy
import pygad
from hybrid_solution_data_loader import get_data
import random
import numpy as np

Environment Definition

In [3]:
# NOTE 1: in this dataset, all jobs can be processed on any workstation
# NOTE 2: Assuming every recipe is ordered exactly once
n_workstations, recipes, operation_times = get_data(0) # 0-3
worst_case = 0
for operation in operation_times:
    max = -float('inf')
    for duration in operation:
        if duration > max:
            max = duration
    worst_case += max
print(f'Worst possible schedule duration: {worst_case}')
last_slot = worst_case

Worst possible schedule duration: 130


In [4]:
def read_data(n_workstations, recipes, operation_times):
    # assuming every recipe gets ordered exactly once with this dataset
    input_format = [0 for _ in range(len(operation_times) * 2)]
    orders = [i for i in range(len(recipes))]
    return input_format, orders, None
    
def translate():
    pass

In [ ]:
input_format, orders, instance = read_data(n_workstations, recipes, operation_times)
# skip translation into own format for now
#recipes, workstations, resources, tasks, orders_model = translate(instance, orders)
#env = SimulationEnvironment(workstations, tasks, resources, recipes)

Helper Functions

In [ ]:
def get_task_for_job(index): # needed?
    return 0 # TODO

def operation_index_to_recipe(index):
    operation_sum = 0
    for i in range(len(recipes)):
        if operation_sum + recipes[i] > index:
            return i
        else:
            operation_sum += recipes[i]
    return -1

def similarity(a, b):
    # used encoding: <assignments, sequence> 
    distance = 0
    pivot = int(len(a)/2)
    for i in range(0, pivot):
        if a[i] != b[i]: # workstation assignment
            distance += len(n_workstations) # get amount of alternative workstations for the respective job
        if a[pivot+i] != b[pivot+i]: # job sequence
            distance += 1 
    return distance

def validate_sequence(individual, order_job_amounts):
    counted = copy.deepcopy(order_job_amounts)
    for i in range(len(individual)/2, len(individual)):
        counted[individual[i]] -= 1
    return all(x == 0 for x in counted)

def validate_assignments(individual):
    return all(individual[i] < n_workstations and individual[i] >= 0 for i in range(len(individual)/2))

def construct_schedule(individual):
    # for each job:
    # check if job is first for it's order
    # start_o = 0
    # else
    # start_o = end of previous job for order
    # check if job is first on workstation
    # start_w = 0
    # else
    """# check for gaps between previous jobs on workstations (between end of previous job and start_o)
    # if any gap is big enough
    # start_w = end of job prior to gap
    # else""" # shouldn't be necessary
    # start_w = end of previous job on workstation
    # set start point as max(start_o, start_w)
    pass

Step 1 - GA: global exploration to find promising areas in the search space

In [ ]:
def mutation_function(offsprings, ga_instance):
    # randomly choose new alternative workstation for assignments
    # randomly switch order for sequence
    pass

def crossover_function(parents, offspring_size, ga_instance):
    # uniform crossover for assignments
    parent1 = parents[0].copy()
    parent2 = parents[1].copy()
    offspring_assignemnts = []
    for i in range(len(operation_times)): # assumption that every recipe is ordered exactly once
        if random.random() < 0.5:
            offspring_assignemnts.append(parent1[i])
        else:
            offspring_assignemnts.append(parent2[i])
    offspring_sequence = []
    # iPOX crossover for sequence
    for i in range(len(operation_times) / 2, 2 * len(operation_times)): # assumption that every recipe is ordered exactly once
        pass
    # -> select parents
    # -> randomly generate 2 job subsets
    # -> copy genes belonging to the jobs in the subsets
    # -> fill in the gaps in order with the rest
    offspring = np.concatenate([offspring_assignemnts, offspring_sequence]) # concats 2 python arrays, returns np array
    return offspring

def selection_function(fitness, num_parents, ga_instance):
    fitness_sorted = sorted(range(len(fitness)), key=lambda k: fitness[k])
    fitness_sorted.reverse()

    parents = np.empty((num_parents, ga_instance.population.shape[1]))
    # for each individual 
    # FN = sum ( 1 / (Cmax(individual) * neighbourhood) )
    # 
    # selection_probability_k = ( 1 / (Cmax(individual) * neighbourhood) ) / FN # selection probability for each individual
    return parents, fitness_sorted[:num_parents] # replace with actual fitness of chosen parents

def fitness_function(solution, solution_idx):
    if validate_sequence(None, None) and validate_assignments(None, None):
        # feasible solution
        # 1 / Cmax(individual) #
        pass
    else:
        # infeasible solution
        pass
    pass

In [ ]:
tasks_jobs = [] # list of tasks in order for lookups with the job id
assignments = []
sequence = []

num_generations = 1000
num_parents_mating = 50
sol_per_pop = 100
init_range_low = 0
init_range_high = len(orders) # doesn't really matter
keep_parents = 10
mutation_percent_genes = 10 # doesn't really matter

crossover_type = crossover_function
mutation_type = mutation_function
fitness_func = fitness_function
parent_selection_type = selection_function

num_genes = len(input_format) * 2
gene_type = int
gene_space = []

for i in range(num_genes / 2):
    # set space for the first half (assignments)
    gene_space.append(len(n_workstations))
for i in range(num_genes / 2, num_genes):
    # set space for the second half (sequence)
    gene_space.append(len(orders))

ga_instance = pygad.GA(num_generations=num_generations, num_parents_mating=num_parents_mating, fitness_func=fitness_func, sol_per_pop=sol_per_pop, num_genes=num_genes, init_range_low=init_range_low, init_range_high=init_range_high, parent_selection_type=parent_selection_type, keep_parents=keep_parents, crossover_type=crossover_type, mutation_type=mutation_type, mutation_percent_genes=mutation_percent_genes, gene_type=gene_type, gene_space=gene_space)
ga_instance.run()
solution, solution_fitness, solution_idx = ga_instance.best_solution()
print("Parameters of the best solution : {solution}".format(solution=solution))
print("Fitness value of the best solution = {solution_fitness}".format(solution_fitness=abs(solution_fitness) - 1))

Intermediate Step: Clustering of the final GA population

Step 2 - Tabu Search: find best individual solutions through local searches

In [ ]:
def get_neighbours(individual):
    pass

def calculate_fitness(individual):
    pass

def search(individuals):
    best = individuals[0]
    max_tabu_size = int(len(individuals) / 4) # for testing
    best_candidate = individuals[0]
    best_fitness = best_candidate_fitness = calculate_fitness(best_candidate)
    tabu_list = []
    tabu_list.append(individuals[0])
    stop = False
    max_iter = 1000
    i = 0
    while not stop:
        neighbours = get_neighbours(best_candidate)
        best_candidate = neighbours[0]
        for candidate in neighbours:
            if candidate not in tabu_list:
                candidate_fitness = calculate_fitness(candidate)
                if candidate_fitness < best_candidate_fitness:
                    best_candidate = candidate
                    best_candidate_fitness = candidate_fitness
        if best_candidate_fitness < best_fitness:
            best = best_candidate
            best_fitness = best_candidate_fitness
        tabu_list.append(best_candidate)
        if len(tabu_list) > max_tabu_size:
            tabu_list.pop()
        stop = i >= max_iter
        i+=1
    return best